In [1]:
import pandas as pd
import random
import datetime

# データの期間を設定
start_date = datetime.date(2022, 1, 31)
end_date = datetime.date(2023, 11, 30)

# 100銘柄のISINを生成
isin_list = ['ISIN{:03}'.format(i) for i in range(1, 1001)]

# データを格納するための空のリストを作成
data = []

# 同じISINで同じクーポン率と時価を生成
coupon_rates = {isin: random.uniform(0.01, 0.10) for isin in isin_list}
market_prices = {isin: random.uniform(80, 120) for isin in isin_list}

# 月ごとのデータを生成
current_date = start_date
while current_date <= end_date:
    for isin in isin_list:
        oas = random.uniform(0.1, 2.0)  # OASをランダムに生成
        t_spread = random.uniform(0.05, 1.0)  # T-Spreadをランダムに生成
        z_spread = random.uniform(0.2, 3.0)  # Z-Spreadをランダムに生成
        remaining_years = random.uniform(0.1, 5.0)  # 残存年数をランダムに設定
        maturity_date = current_date.replace(year=current_date.year + 1)  # 償還日を1年後に設定
        payment_frequency = random.choice([1, 2, 4])  # 利払い回数をランダムに選択
        coupon_rate = coupon_rates[isin]  # 同じISINのクーポン率を取得
        market_price = market_prices[isin]  # 同じISINの時価を取得

        # データをリストに追加
        data.append([current_date, isin, oas, t_spread, z_spread, remaining_years, maturity_date, payment_frequency, coupon_rate, market_price])

    # 月を進める
    if current_date.month == 12:
        current_date = current_date.replace(year=current_date.year + 1, month=1, day=1)
    else:
        current_date = current_date.replace(month=current_date.month + 1, day=1)

# データフレームを作成
columns = ['日付', 'ISIN', 'OAS', 'T-Spread', 'Z-Spread', '残存年数', '償還日', '利払い回数', 'クーポン率', '時価']
df = pd.DataFrame(data, columns=columns)

# データフレームを表示
print(df)

# データフレームを表示
df.to_csv("test_df.csv", encoding = "shift-jis")

               日付      ISIN       OAS  T-Spread  Z-Spread      残存年数  \
0      2022-01-31   ISIN001  1.732160  0.326391  2.527765  3.324283   
1      2022-01-31   ISIN002  0.421740  0.680694  2.415524  2.655772   
2      2022-01-31   ISIN003  1.492336  0.507984  2.169990  4.439098   
3      2022-01-31   ISIN004  1.155965  0.491049  1.308431  1.377327   
4      2022-01-31   ISIN005  0.202505  0.452545  2.262269  2.997527   
...           ...       ...       ...       ...       ...       ...   
22995  2023-11-01   ISIN996  0.454829  0.130488  1.835894  3.058863   
22996  2023-11-01   ISIN997  1.297955  0.798839  1.902458  0.525119   
22997  2023-11-01   ISIN998  1.671543  0.605166  1.331776  1.539639   
22998  2023-11-01   ISIN999  1.911004  0.655597  2.241086  3.869343   
22999  2023-11-01  ISIN1000  0.632961  0.549521  1.611275  4.686144   

              償還日  利払い回数     クーポン率          時価  
0      2023-01-31      4  0.079923   86.333904  
1      2023-01-31      1  0.053870   97.110132  


In [2]:
df_test = pd.read_csv("test_df.csv", encoding = "shift-jis", index_col = 0)
sub_dict = {
    "coupon_rate" : "クーポン率",
    "price" : "時価",
    "redemption‗date" : "償還日",
    "payment_num" : "利払い回数"
}

In [30]:
import pandas as pd

def calculate_returns(df, sub_dict):
    # トータルリターンと累積リターンの列を初期化
    df['トータルリターン'] = 0.0
    df['累積リターン'] = 1.0  # 初期の累積リターンは1.0
    
    # ISINごとに累積リターンを計算
    cumulative_returns = {}
    
    for isin, group in df.groupby('ISIN'):
        prev_market_price = None  # 前月の市場価格を保持する変数
        
        for index, row in group.iterrows():
            # 償還日を文字列から日付オブジェクトに変換
            maturity_date = pd.to_datetime(row[sub_dict["redemption‗date"]])
            current_date = pd.to_datetime(row['日付'])
            
            # クーポンが発生したかどうかの判定
            months_until_maturity = (maturity_date.year - current_date.year) * 12 + maturity_date.month - current_date.month
            coupon_generated = months_until_maturity % row[sub_dict["payment_num"]] == 0
            
            # 前月の市場価格を使用してトータルリターンを計算
            if prev_market_price is not None:
                price_change = row[sub_dict["price"]] - prev_market_price
                coupon_income = 100 * row[sub_dict["coupon_rate"]] / row[sub_dict["payment_num"]] # 元本は100として処理
            else:
                price_change = 0.0
                coupon_income = 0.0
            
            # ゼロで割り算を防ぐため、前月の市場価格がNoneの場合、デフォルトの値1.0を使用
            prev_market_price = prev_market_price or 1.0
            
            total_return = (coupon_income + price_change) / prev_market_price  # トータルリターン
            
            # 累積リターンの計算
            cumulative_return = df.at[index, '累積リターン'] * (1 + total_return)
            
            # トータルリターンと累積リターンをデータフレームに設定
            df.at[index, 'トータルリターン'] = total_return
            df.at[index, '累積リターン'] = cumulative_return
            
            prev_market_price = row['時価']  # 現在の市場価格を前月の値として保存
    
    df = df.reset_index(drop = True)

    return df, cumulative_returns

In [31]:
df_return, cumulative_returns = calculate_returns(df_test) # , sub_dict

In [32]:
df_return.to_csv("test_return.csv", encoding = "shift-jis")

In [22]:
import pandas as pd

# 例としてのデータフレーム
df = pd.DataFrame({
    'Category': ['A', 'B', 'A', 'C', 'B', 'C', 'A'],
    'Value': [10, 20, 15, 30, 25, 35, 40]
})

# 'Category' カラムに基づいてグループ化
grouped = df.groupby('Category')

# 各グループに対して何か操作を行う
for name, group in grouped:
    print(f"Category: {name}")
    print(group)

Category: A
  Category  Value
0        A     10
2        A     15
6        A     40
Category: B
  Category  Value
1        B     20
4        B     25
Category: C
  Category  Value
3        C     30
5        C     35


In [56]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime

# 50個のユニークなISINコードを生成
unique_isins = ['ISIN' + str(i).zfill(3) for i in range(1, 51)]

# 信用格付けのリスト
credit_ratings = ['AAA', 'BBB', 'CCC', 'DDD']

# ベースとなる日付をdatetimeオブジェクトに変換
base_date = datetime.strptime('2023-01-01', '%Y-%m-%d')

# 100時点分のデータセットを生成
num_points = 100
data_list = []

# 各ISINコードに対してデータを生成
for isin in unique_isins:
    for day_offset in range(num_points):
        current_date = base_date + timedelta(days=day_offset)
        bond_price = 100 + np.random.randint(-2, 3)  # ボンドの価格をランダムに変動させる
        coupon_rate = 5  # 一定のクーポンレート
        credit_rating = np.random.choice(credit_ratings)
        data_list.append({
            'Date': current_date,
            'ISIN': isin,
            'Bond_Price': bond_price,
            'Coupon_Rate': coupon_rate,
            'Credit_Rating': credit_rating
        })

# 生成されたデータフレームを結合
df = pd.DataFrame(data_list)

# 最初のいくつかのデータを表示
df


,Date,ISIN,Bond_Price,Coupon_Rate,Credit_Rating
0,2023-01-01,ISIN001,102,5,AAA
1,2023-01-02,ISIN001,101,5,DDD
2,2023-01-03,ISIN001,101,5,BBB
3,2023-01-04,ISIN001,102,5,CCC
4,2023-01-05,ISIN001,101,5,DDD
...,...,...,...,...,...
4995,2023-04-06,ISIN050,98,5,CCC
4996,2023-04-07,ISIN050,98,5,DDD
4997,2023-04-08,ISIN050,102,5,BBB
4998,2023-04-09,ISIN050,101,5,CCC


In [57]:
import pandas as pd

def calculate_total_return(df):
    # 前時点との債券価格の差分を計算
    df['Price_Diff'] = df['Bond_Price'].diff().fillna(0)
    # インカムゲインの計算
    df['Income_Gain'] = (df['Coupon_Rate'] / 100) * 100  # 額面価格100と仮定
    # トータルリターンの計算
    df['Total_Return'] = ((df['Price_Diff'] + df['Income_Gain']) / df['Bond_Price'].shift())
    # 
    df['Total_Return'].fillna(0, inplace=True)
    
    return df

# ISINコードごとにグループ化し、各グループに対してトータルリターンを計算
df = df.groupby('ISIN').apply(calculate_total_return).reset_index(drop = True)

df.to_csv("test.csv")

In [58]:
# 累積リターンを計算する関数
def calculate_cumulative_return(df):
    df['Cumulative_Return'] = df.groupby('ISIN')['Total_Return'].cumsum()
    return df

# 累積リターンを計算
df = calculate_cumulative_return(df).reset_index(drop = True)

df

,Date,ISIN,Bond_Price,Coupon_Rate,Credit_Rating,Price_Diff,Income_Gain,Total_Return,Cumulative_Return
0,2023-01-01,ISIN001,102,5,AAA,0.0,5.0,0.000000,0.000000
1,2023-01-02,ISIN001,101,5,DDD,-1.0,5.0,0.039216,0.039216
2,2023-01-03,ISIN001,101,5,BBB,0.0,5.0,0.049505,0.088721
3,2023-01-04,ISIN001,102,5,CCC,1.0,5.0,0.059406,0.148127
4,2023-01-05,ISIN001,101,5,DDD,-1.0,5.0,0.039216,0.187342
...,...,...,...,...,...,...,...,...,...
4995,2023-04-06,ISIN050,98,5,CCC,0.0,5.0,0.051020,4.745981
4996,2023-04-07,ISIN050,98,5,DDD,0.0,5.0,0.051020,4.797001
4997,2023-04-08,ISIN050,102,5,BBB,4.0,5.0,0.091837,4.888838
4998,2023-04-09,ISIN050,101,5,CCC,-1.0,5.0,0.039216,4.928054


In [61]:
# 判定に使用するカラム名のリスト
columns_to_check = ['A', 'B', 'C']

def check_green_flag(columns_to_check):
    # 新しいカラム 'merged_green_flag' を追加
    df['merged_green_flag'] = df[columns_to_check].max(axis=1)

df

,Date,ISIN,Bond_Price,Coupon_Rate,Credit_Rating,Price_Diff,Income_Gain,Total_Return,Cumulative_Return
0,2023-01-01,ISIN001,102,5,AAA,0.0,5.0,0.000000,0.000000
1,2023-01-02,ISIN001,101,5,DDD,-1.0,5.0,0.039216,0.039216
2,2023-01-03,ISIN001,101,5,BBB,0.0,5.0,0.049505,0.088721
3,2023-01-04,ISIN001,102,5,CCC,1.0,5.0,0.059406,0.148127
4,2023-01-05,ISIN001,101,5,DDD,-1.0,5.0,0.039216,0.187342
...,...,...,...,...,...,...,...,...,...
4995,2023-04-06,ISIN050,98,5,CCC,0.0,5.0,0.051020,4.745981
4996,2023-04-07,ISIN050,98,5,DDD,0.0,5.0,0.051020,4.797001
4997,2023-04-08,ISIN050,102,5,BBB,4.0,5.0,0.091837,4.888838
4998,2023-04-09,ISIN050,101,5,CCC,-1.0,5.0,0.039216,4.928054


In [64]:
# 信用格付けと日付ごとにグループ化し、平均値を計算
average_return = df.groupby(['Credit_Rating', 'Date'])["Cumulative_Return"].mean().reset_index()

In [65]:
average_return

,Credit_Rating,Date,Cumulative_Return
0,AAA,2023-01-01,0.000000
1,AAA,2023-01-02,0.051660
2,AAA,2023-01-03,0.109695
3,AAA,2023-01-04,0.158381
4,AAA,2023-01-05,0.209132
...,...,...,...
395,DDD,2023-04-06,4.769021
396,DDD,2023-04-07,4.819908
397,DDD,2023-04-08,4.873830
398,DDD,2023-04-09,4.924756


In [ ]:
def calculate_returns_improved(df, sub_dict):
    # トータルリターンと累積リターンの列を初期化
    df['トータルリターン'] = 0.0
    df['累積リターン'] = 1.0  # 初期の累積リターンは1.0
    
    # ISINごとに累積リターンを計算
    cumulative_returns = {}
    
    for isin, group in df.groupby('ISIN'):
        prev_market_price = None  # 前月の市場価格を保持する変数
        
        for index, row in group.iterrows():
            # 償還日を文字列から日付オブジェクトに変換
            maturity_date = pd.to_datetime(row[sub_dict["redemption‗date"]])
            current_date = pd.to_datetime(row['日付'])
            
            # クーポンが発生したかどうかの判定
            months_until_maturity = (maturity_date.year - current_date.year) * 12 + maturity_date.month - current_date.month
            coupon_generated = months_until_maturity % row[sub_dict["payment_num"]] == 0
            
            # 前月の市場価格を使用してトータルリターンを計算
            if prev_market_price is None:
                prev_market_price = row[sub_dict["price"]]  # 最初のループのため現在の行の市場価格を使用
                continue  # 最初のループでは計算をスキップ

            price_change = row[sub_dict["price"]] - prev_market_price
            coupon_income = (100 * row[sub_dict["coupon_rate"]] / row[sub_dict["payment_num"]]) if coupon_generated else 0.0
            
            total_return = (coupon_income + price_change) / prev_market_price  # トータルリターン

            # 累積リターンの計算
            if isin in cumulative_returns:
                cumulative_returns[isin] *= (1 + total_return)
            else:
                cumulative_returns[isin] = 1 + total_return
            
            # トータルリターンと累積リターンをデータフレームに設定
            df.at[index, 'トータルリターン'] = total_return
            df.at[index, '累積リターン'] = cumulative_returns[isin]
            
            prev_market_price = row[sub_dict["price"]]  # 現在の市場価格を前月の値として保存
    
    return df

# この関数は、入力されたDataFrame 'df' と 'sub_dict'（列名と実際の列をマッピングする辞書）を使用して、各ISINに対するトータルリターンと累積リターンを計算します。


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib.backends.backend_pdf import PdfPages

def analyze_and_plot_data(data_df, method_column, num_groups, output_filename):
    # 指定した方法（カテゴリ）でデータを分割
    groups = []
    for _, group_df in data_df.groupby(method_column):
        groups.append(group_df)

    # PDFファイルに分布図を出力
    pdf = PdfPages(output_filename)

    for i, group_df in enumerate(groups):
        plt.figure(figsize=(8, 4))
        plt.hist(group_df['リターン'], bins=20, density=True, alpha=0.6, label=f'Group {i + 1}')
        plt.title(f'Distribution of Returns for Group {i + 1}')
        plt.xlabel('Return')
        plt.ylabel('Frequency')
        plt.legend()
        pdf.savefig()
        plt.close()

    pdf.close()

    # 分位点を計算してエクセルに出力
    quantiles = np.linspace(0, 100, num=num_groups + 1)[1:-1]  # 分割数に合わせた分位点を計算

    quantile_data = []
    for group_df in groups:
        group_quantiles = np.percentile(group_df['リターン'], quantiles)
        for q, quantile_value in zip(quantiles, group_quantiles):
            quantile_data.append({'カテゴリ': group_df[method_column].iloc[0],
                                  f'{q}%': quantile_value})

    quantile_df = pd.DataFrame(quantile_data)

    # エクセルに出力
    excel_output_filename = f'{method_column}_quantiles.xlsx'
    quantile_df.to_excel(excel_output_filename, index=False)

    print("Quantile Data:")
    print(quantile_df)

# 信用格付けデータの読み込み（適切なデータを読み込んでください）
# この例では適当なデータを仮定します
data = {
    'ISIN': ['ISIN001', 'ISIN002', 'ISIN003', 'ISIN004', 'ISIN005', 'ISIN006'],
    'リターン': [0.05, 0.1, 0.08, 0.06, 0.12, 0.09],
    '信用格付け': ['AAA', 'AA', 'BBB', 'AAA', 'BB', 'AA'],
}
df = pd.DataFrame(data)

# 分割数を指定して関数を呼び出してデータを分析し、プロット
analyze_and_plot_data(df, '信用格付け', 3, 'credit_rating_analysis.pdf')



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def analyze_and_plot_data(data_df, method_column, num_groups, output_filename):
    # 指定した方法（カテゴリ）でデータを分割
    groups = []
    for _, group_df in data_df.groupby(method_column):
        groups.append(group_df)

    # PDFファイルに散布図を出力
    pdf = PdfPages(output_filename)

    for i, group_df in enumerate(groups):
        plt.figure(figsize=(8, 4))
        plt.scatter(group_df[method_column], group_df['リターン'], alpha=0.6,
                    label=f'Group {i + 1}')
        plt.title(f'Scatter Plot of Returns for Group {i + 1}')
        plt.xlabel(method_column)
        plt.ylabel('Return')
        plt.legend()
        pdf.savefig()
        plt.close()

    pdf.close()

# 信用格付けデータの読み込み（適切なデータを読み込んでください）
# この例では適当なデータを仮定します
data = {
    'ISIN': ['ISIN001', 'ISIN002', 'ISIN003', 'ISIN004', 'ISIN005', 'ISIN006'],
    'リターン': [0.05, 0.1, 0.08, 0.06, 0.12, 0.09],
    '信用格付け': ['AAA', 'AA', 'BBB', 'AAA', 'BB', 'AA'],
}
df = pd.DataFrame(data)

# 分割数を指定して関数を呼び出してデータを分析し、プロット
analyze_and_plot_data(df, '信用格付け', 3, 'credit_rating_analysis.pdf')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def analyze_and_plot_data(data_df, method_column, num_groups, output_filename, flag_column):
    # 指定した方法（カテゴリ）でデータを分割
    groups = []
    for _, group_df in data_df.groupby(method_column):
        groups.append(group_df)

    # 分位点を計算
    quantiles = np.linspace(0, 100, num=num_groups + 1)[1:-1]  # 分割数に合わせた分位点を計算

    with pd.ExcelWriter(output_filename) as writer:
        for i, group_df in enumerate(groups):
            quantile_data = {'カテゴリ': group_df[method_column].iloc[0]}
            
            # フラグが付いているデータと付いていないデータを分けて分位点を計算
            flagged_quantiles = np.percentile(group_df[group_df[flag_column]]['リターン'], quantiles)
            non_flagged_quantiles = np.percentile(group_df[~group_df[flag_column]]['リターン'], quantiles)
            
            for q, flagged, non_flagged in zip(quantiles, flagged_quantiles, non_flagged_quantiles):
                quantile_data[f'{q}% (Flagged)'] = flagged
                quantile_data[f'{q}% (Non-Flagged)'] = non_flagged
            
            quantile_df = pd.DataFrame([quantile_data])
            
            # シート名に銘柄数を含める
            num_securities = len(group_df)
            sheet_name = f'Sheet {i + 1} ({num_securities} Securities)'
            quantile_df.to_excel(writer, sheet_name=sheet_name, index=False)

# 信用格付けデータの読み込み（適切なデータを読み込んでください）
# この例では適当なデータを仮定します
data = {
    'ISIN': ['ISIN001', 'ISIN002', 'ISIN003', 'ISIN004', 'ISIN005', 'ISIN006'],
    'リターン': [0.05, 0.1, 0.08, 0.06, 0.12, 0.09],
    '信用格付け': ['AAA', 'AA', 'BBB', 'AAA', 'BB', 'AA'],
    'フラグ': [True, False, True, False, True, False],  # フラグが付いているかどうかを示す列
}
df = pd.DataFrame(data)

# 分割数を指定して関数を呼び出してデータを分析し、Excelファイルに出力
analyze_and_plot_data(df, '信用格付け', 3, 'credit_rating_quantiles.xlsx', 'フラグ')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def analyze_and_plot_data(data_df, method_column, num_groups, output_filename, flag_column):
    # 分位点を計算
    quantiles = np.linspace(0, 100, num=num_groups + 1)

    # PDFファイルに散布図を出力
    pdf = PdfPages(output_filename)

    for i in range(1, num_groups + 1):
        lower_percentile = quantiles[i - 1]
        upper_percentile = quantiles[i]

        # 分位点に基づいてデータを抽出
        lower_quantile_value = np.percentile(data_df[method_column], lower_percentile)
        upper_quantile_value = np.percentile(data_df[method_column], upper_percentile)
        subset_data = data_df[(data_df[method_column] >= lower_quantile_value) & (data_df[method_column] <= upper_quantile_value)]

        # フラグが付いているデータと付いていないデータを分けてプロット
        flagged_data = subset_data[subset_data[flag_column]]
        non_flagged_data = subset_data[~subset_data[flag_column]]

        plt.figure(figsize=(8, 4))
        plt.scatter(flagged_data[method_column], flagged_data['リターン'], alpha=0.6, c='red',
                    label=f'Flagged: {lower_percentile:.0f}%~{upper_percentile:.0f}%')
        plt.scatter(non_flagged_data[method_column], non_flagged_data['リターン'], alpha=0.6, c='blue',
                    label=f'Non-Flagged: {lower_percentile:.0f}%~{upper_percentile:.0f}%')
        plt.title(f'Scatter Plot of Returns for Percentile {lower_percentile:.0f}%~{upper_percentile:.0f}%')
        plt.xlabel(method_column)
        plt.ylabel('Return')
        plt.legend()
        pdf.savefig()
        plt.close()

    pdf.close()

# 信用格付けデータの読み込み（適切なデータを読み込んでください）
# この例では適当なデータを仮定します
data = {
    'ISIN': ['ISIN001', 'ISIN002', 'ISIN003', 'ISIN004', 'ISIN005', 'ISIN006'],
    'リターン': [0.05, 0.1, 0.08, 0.06, 0.12, 0.09],
    '信用格付け': ['AAA', 'AA', 'BBB', 'AAA', 'BB', 'AA'],
    'フラグ': [True, False, True, False, True, False],  # フラグが付いているかどうかを示す列
}
df = pd.DataFrame(data)

# 分割数を指定して関数を呼び出してデータを分析し、プロット
analyze_and_plot_data(df, '信用格付け', 3, 'credit_rating_analysis.pdf', 'フラグ')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def analyze_and_plot_data(data_df, method_column, num_groups, output_filename, flag_column):
    # 月ごとにデータを分割
    data_df['日付'] = pd.to_datetime(data_df['日付'])
    data_df['月'] = data_df['日付'].dt.month

    # PDFファイルに散布図を出力
    pdf = PdfPages(output_filename)

    # 1月から12月までの各月について処理
    for month in range(1, 13):
        # 月ごとのデータを抽出
        monthly_data = data_df[data_df['月'] == month]

        # サブプロットの行数と列数を計算
        num_rows = int(np.sqrt(num_groups))  # データ分割数の平方根を行数とする
        num_cols = (num_groups + num_rows - 1) // num_rows  # 列数を計算

        # 分位点を計算
        quantiles = np.linspace(0, 100, num=num_groups + 1)  # 分割数に合わせた分位点を計算

        # サブプロットのサイズを調整
        fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 8))

        # サブプロットに散布図をプロット
        for i, group_df in enumerate(monthly_data.groupby(method_column)):
            row = i // num_cols
            col = i % num_cols
            lower_percentile = quantiles[i]
            upper_percentile = quantiles[i + 1]

            # フラグが付いているデータと付いていないデータを分けてプロット
            flagged_data = group_df[1][group_df[1][flag_column]]
            non_flagged_data = group_df[1][~group_df[1][flag_column]]

            axes[row, col].scatter(flagged_data[method_column], flagged_data['リターン'], alpha=0.6, c='red',
                                   label=f'Flagged: {lower_percentile:.0f}%~{upper_percentile:.0f}%')
            axes[row, col].scatter(non_flagged_data[method_column], non_flagged_data['リターン'], alpha=0.6, c='blue',
                                   label=f'Non-Flagged: {lower_percentile:.0f}%~{upper_percentile:.0f}%')
            axes[row, col].set_title(f'Percentile {lower_percentile:.0f}%~{upper_percentile:.0f}%')
            axes[row, col].set_xlabel(method_column)
            axes[row, col].set_ylabel('Return')
            axes[row, col].legend()

        # PDFに保存
        pdf.savefig(fig)
        plt.close()

    pdf.close()

# 信用格付けデータの読み込み（適切なデータを読み込んでください）
# この例では適当なデータを仮定します
data = {
    'ISIN': ['ISIN001', 'ISIN002', 'ISIN003', 'ISIN004', 'ISIN005', 'ISIN006'],
    'リターン': [0.05, 0.1, 0.08, 0.06, 0.12, 0.09],
    '信用格付け': ['AAA', 'AA', 'BBB', 'AAA', 'BB', 'AA'],
    'フラグ': [True, False, True, False, True, False],  # フラグが付いているかどうかを示す列
    '日付': ['2023-01-01', '2023-01-15', '2023-02-01', '2023-03-01', '2023-03-15', '2023-04-01'],
}
df = pd.DataFrame(data)

# 分割数を指定して関数を呼び出してデータを分析し、プロット
analyze_and_plot_data(df, '信用格付け', 3, 'credit_rating_analysis.pdf', 'フラグ')


In [2]:
import pandas as pd

# 仮のデータフレームを作成（実際のデータに合わせて読み込む）
data = {'Date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04'],
        'ISIN': ['ISIN0001', 'ISIN0001', 'ISIN0002', 'ISIN0001'],
        'Flag1': [False, True, False, False],
        'Flag2': [True, False, False, True],
        'Flag3': [False, False, True, False]}
df = pd.DataFrame(data)
df_ = df.copy()

# 各フラグ列について処理を行う
flag_columns = ['Flag1', 'Flag2', 'Flag3']  # フラグの列名をリストで指定

for flag in flag_columns:
    # ISINごとにグループ化し、各フラグがTrueになるかどうかを判定
    max_flag = df_.groupby('ISIN')[flag].transform('max')
    # 元のデータフレームのフラグ列を更新
    df_[flag] = max_flag

# 結果の表示
print(df)


         Date      ISIN  Flag1  Flag2  Flag3
0  2023-01-01  ISIN0001  False   True  False
1  2023-01-02  ISIN0001   True  False  False
2  2023-01-03  ISIN0002  False  False   True
3  2023-01-04  ISIN0001  False   True  False


In [3]:
df_

,Date,ISIN,Flag1,Flag2,Flag3
0,2023-01-01,ISIN0001,True,True,False
1,2023-01-02,ISIN0001,True,True,False
2,2023-01-03,ISIN0002,False,False,True
3,2023-01-04,ISIN0001,True,True,False


In [4]:
df

,Date,ISIN,Flag1,Flag2,Flag3
0,2023-01-01,ISIN0001,False,True,False
1,2023-01-02,ISIN0001,True,False,False
2,2023-01-03,ISIN0002,False,False,True
3,2023-01-04,ISIN0001,False,True,False
